In [1]:
# Importing essential libraries and functions

import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from numpy import array
import tensorflow as tf

from keras.preprocessing.text import one_hot, Tokenizer
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, GlobalMaxPooling1D, Embedding, Conv1D, LSTM
from sklearn.model_selection import train_test_split

from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
reviews = pd.read_csv(r".\Dataset\yelpReviewsDataset.csv")

In [3]:
def sentiment_to_int(sentiment):
    sentiment_map = {
        1: 0,
        2: 1,
        3: 2,
        4: 3,
        5: 4
    }
    return sentiment_map.get(sentiment, -1)

In [4]:
texts = reviews['Review'].values
labels = reviews['Rating'].apply(sentiment_to_int).values

In [5]:
import pickle
with open(r".\TokenizedDataset\yelpReviewsDatasetTokens.pkl", "rb") as file:
    tokenized_Reviews = pickle.load(file)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(tokenized_Reviews, labels, test_size=0.20, random_state=42)

In [7]:
# One-hot encode labels
y_train = to_categorical(y_train, num_classes=5)
y_test = to_categorical(y_test, num_classes=5)

In [8]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(X_train)

X_train = word_tokenizer.texts_to_sequences(X_train)
X_test = word_tokenizer.texts_to_sequences(X_test)

In [9]:
import io
import json
tokenizer_json = word_tokenizer.to_json()
with io.open('snn_tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

In [10]:
# Adding 1 to store dimensions for words for which no pretrained word embeddings exist
vocab_length = len(word_tokenizer.word_index) + 1
vocab_length

75957

In [11]:
max_length = 1000

In [12]:
# Padding all reviews to fixed length
X_train = pad_sequences(X_train, padding='post', maxlen=max_length)
X_test = pad_sequences(X_test, padding='post', maxlen=max_length)

In [13]:
# Load GloVe word embeddings and create an Embeddings Dictionary
from numpy import asarray
from numpy import zeros
embeddings_dictionary = dict()
glove_file = open('glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

In [14]:
# Create Embedding Matrix having 100 columns 
# Containing 100-dimensional GloVe word embeddings for all words in our corpus.

embedding_matrix = zeros((vocab_length, 100))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [15]:
embedding_matrix.shape

(75957, 100)

Simple Neural Network

In [16]:
# Neural Network architecture
snn_model = Sequential()
embedding_layer = Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=max_length , trainable=False)
snn_model.add(embedding_layer)
snn_model.add(Flatten())
snn_model.add(Dense(5, activation='softmax'))

In [17]:
# Model compiling
snn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
print(snn_model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1000, 100)         7595700   
                                                                 
 flatten (Flatten)           (None, 100000)            0         
                                                                 
 dense (Dense)               (None, 5)                 500005    
                                                                 
Total params: 8,095,705
Trainable params: 500,005
Non-trainable params: 7,595,700
_________________________________________________________________
None


In [18]:
# Model training
snn_model_history = snn_model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1, validation_split=0.2)

Epoch 1/10
1250/1250 [==============================] - 32s 26ms/step - loss: 1.3852 - acc: 0.4259 - val_loss: 1.3639 - val_acc: 0.4481
Epoch 2/10
1250/1250 [==============================] - 34s 27ms/step - loss: 1.2113 - acc: 0.4954 - val_loss: 1.4164 - val_acc: 0.4485
Epoch 3/10
1250/1250 [==============================] - 32s 25ms/step - loss: 1.1656 - acc: 0.5149 - val_loss: 1.4376 - val_acc: 0.4491
Epoch 4/10
1250/1250 [==============================] - 37s 29ms/step - loss: 1.1465 - acc: 0.5242 - val_loss: 1.5087 - val_acc: 0.4448
Epoch 5/10
1250/1250 [==============================] - 33s 26ms/step - loss: 1.1318 - acc: 0.5299 - val_loss: 1.5153 - val_acc: 0.4462
Epoch 6/10
1250/1250 [==============================] - 32s 25ms/step - loss: 1.1167 - acc: 0.5371 - val_loss: 1.5597 - val_acc: 0.4472
Epoch 7/10
1250/1250 [==============================] - 28s 22ms/step - loss: 1.1079 - acc: 0.5396 - val_loss: 1.5990 - val_acc: 0.4486
Epoch 8/10
1250/1250 [==========================

In [19]:
# Predictions on the Test Set
score = snn_model.evaluate(X_test, y_test, verbose=1)
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

1563/1563 [==============================] - 7s 5ms/step - loss: 1.7190 - acc: 0.4460
Test Score: 1.7189948558807373
Test Accuracy: 0.4459800124168396


In [20]:
#Predict
y_prediction = snn_model.predict(X_test)
y_prediction = np.argmax(y_prediction, axis = 1)
y_test=np.argmax(y_test, axis=1)
#Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(y_test, y_prediction , normalize='pred')
print(result)

1563/1563 [==============================] - 6s 4ms/step
[[0.58652793 0.27525387 0.11007752 0.03352404 0.03295374]
 [0.25520263 0.38464279 0.2514581  0.10739737 0.0566361 ]
 [0.09611172 0.218956   0.32942045 0.22954933 0.09160536]
 [0.03030303 0.07999287 0.2118863  0.37488708 0.26282374]
 [0.03185469 0.04115446 0.09715762 0.25464218 0.55598105]]


In [21]:
accuracy_score = metrics.accuracy_score(y_prediction, y_test)

print('SNN accuracy is',str('{:04.2f}'.format(accuracy_score*100))+'%')
print('------------------------------------------------')
print('Confusion Matrix:')
print(pd.DataFrame(confusion_matrix(y_test, y_prediction)))
print('------------------------------------------------')
print('Classification Report:')
print(classification_report(y_test, y_prediction))

SNN accuracy is 44.60%
------------------------------------------------
Confusion Matrix:
      0     1     2     3     4
0  6426  1545  1491   334   327
1  2796  2159  3406  1070   562
2  1053  1229  4462  2287   909
3   332   449  2870  3735  2608
4   349   231  1316  2537  5517
------------------------------------------------
Classification Report:
              precision    recall  f1-score   support

           0       0.59      0.63      0.61     10123
           1       0.38      0.22      0.28      9993
           2       0.33      0.45      0.38      9940
           3       0.37      0.37      0.37      9994
           4       0.56      0.55      0.56      9950

    accuracy                           0.45     50000
   macro avg       0.45      0.45      0.44     50000
weighted avg       0.45      0.45      0.44     50000

